# Let's Evaluate SmolLM





### Configure the machine and install packages


In [10]:
from typing import Literal
import jax

platform : Literal["darwin", "colab", "cuda", "tpu"] = "darwin"

try:
    import google.colab
    platform = "colab"
except ImportError:
    devices = jax.devices()
    if any(d.platform == "gpu" for d in devices):
        platform = "cuda"
    if any(d.platform == "tpu" for d in devices):
        platform = "tpu"

print(f"Running on {platform}")

if platform == "colab":
    !git clone https://github.com/novastar53/jaxpt
    !cd jaxpt && git checkout main && git pull
    !pip install tiktoken datasets --quiet
    #!pip uninstall -y tensorflow

Running on colab
fatal: destination path 'jaxpt' already exists and is not an empty directory.
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.


In [11]:
from pathlib import Path
import sys

if platform == "colab":
    jaxpt_dir = str(Path().absolute() / "jaxpt" / "src" )
else:
    jaxpt_dir = str(Path().absolute().parent / "src" )


sys.path.append(jaxpt_dir)
print(jaxpt_dir)

/content/jaxpt/src


In [12]:
import os


import warnings

import jax
from jax.sharding import Mesh, PartitionSpec, NamedSharding
import flax
import jax.numpy as jnp
import numpy as np

# Hardware setup
print("JAX version:", jax.__version__)
print("Flax version", flax.__version__)

devices = jax.devices()
num_devices = len(devices)
print("Available devices:", num_devices)

requested_device = "gpu"

jax.config.update("jax_platform_name", requested_device) # Make sure we're using the GPU

device = jax.default_backend()
if device != requested_device:
    warnings.warn(f"not using {requested_device}. Using {device}")
else:
    print(f"using {device}")


#####################################
##        jax.lax matmul presets   ##
#####################################
## 'ANY_F8_ANY_F8_F32',
## 'ANY_F8_ANY_F8_F32_FAST_ACCUM'
## 'ANY_F8_ANY_F8_ANY'
## 'ANY_F8_ANY_F8_ANY_FAST_ACCUM'
## 'F16_F16_F16'
## 'F16_F16_F32'
## 'BF16_BF16_BF16'
## 'BF16_BF16_F32'
## 'BF16_BF16_F32_X3'
## 'BF16_BF16_F32_X6'
## 'TF32_TF32_F32'
## 'TF32_TF32_F32_X3'
## 'F32_F32_F32'
## 'F64_F64_F64'
#####################################

jax.config.update("jax_default_matmul_precision", "BF16_BF16_F32") # Set the default precision for matrix multiplication

#jax.config.update("jax_enable_x64", True) # Make sure the highest precision is enabled in case we need
#os.environ["NVIDIA_TF32_OVERRIDE"] = "1"
#os.environ["JAX_ENABLE_X64"] = "False"

if device == "tpu":
    def list_tpu_memory():
        devices = jax.devices()
        for device in devices:
            if 'TPU' in str(device.device_kind):
                print(f"Device: {device}, Memory: {device.memory_stats()['bytes_limit']/(1024*1024)},  Used: {device.memory_stats()['bytes_in_use']/(1024*1024)}")

    list_tpu_memory()

# Set up device mesh


mesh = jax.sharding.Mesh(jax.devices(), ["devices"])

# Test the device
A = jnp.array(np.random.normal(size=(4096, 4096)), dtype=jnp.float32) # Make sure matmul is fast
%timeit (A@A).block_until_ready()

JAX version: 0.5.2
Flax version 0.10.6
Available devices: 1
using gpu
3.3 ms ± 66.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
import random
from datetime import datetime
import string
from jaxpt.checkpointers import save_checkpoint, load_checkpoint, load_checkpoint_from_gcloud

def generate_random_code(length=6):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

timestamp = datetime.now().strftime("%Y%m%d")
random_code = generate_random_code()

run_dirname = f"run_{timestamp}_{random_code}"
print(f"Run: {run_dirname}")

if platform == "cuda":
  output_dir = Path("/home/ubuntu/alpha_training_runs") # Lambda Labs setup
else:
  output_dir = Path().absolute().parent  / "alpha_training_runs" # Local setup

Run: run_20250722_plsbad


In [14]:
from flax import nnx

from jaxpt.infer import generate_completions, generate
from jaxpt.models import SmolLM, SmolLM_Config
from jaxpt.models.smol_lm import from_hf_pretrained
from jaxpt.utils import count_params, create_sharded_model

import tiktoken
from transformers import AutoTokenizer

key = jax.random.PRNGKey(1337)
rngs = nnx.Rngs(key)
config = SmolLM_Config(
                     name="HuggingFaceTB/SmolLM-135M",
                     dtype=jnp.bfloat16, \
                     vocab_size=49152,
                     n_layer=30,
                     block_size=2048,
                     n_head=9,
                     n_kv_head=3,
                     n_mlp_hidden=1536,
                     sdpa_implementation="cudnn" if device=="gpu" else "xla")
nnx.display(config)

with mesh:
    #m = create_sharded_model(Tiny_MoE, config, rngs)
    #m = load_checkpoint(Tiny_MoE, output_dir, config, "run_20250716_htlpzj", 180000, rngs)
    #m = load_checkpoint(Tiny_MoE, output_dir, config, "run_20250716_htlpzj", 381469, rngs)
    m = from_hf_pretrained(config, rngs)
    #m = load_hf_pretrained()

    graphdef, rngstate, state = nnx.split(m, nnx.RngState, ...)
    total_params = count_params(m)
    moe_params = count_params(m, "moe")

    print(f"Parameter Count: {total_params:,}")
    print(f"MOE Parameter Count: {moe_params:,}")
    print(f"Replicated Parameter Count: {total_params - moe_params:,}")
    nnx.display(state)

Parameter Count: 134,515,008
MOE Parameter Count: 0
Replicated Parameter Count: 134,515,008


In [15]:
with mesh:
    enc = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
    completions = generate_completions(m, enc=enc, num_completions=8,
                                    max_length=21,
                                    prefix="Once upon a time, there lived a")
    for completion in completions:
        print(completion)

Once upon a time, there lived a curious little girl named Lily. She loved exploring her neighborhood and learning
Once upon a time, there lived a curious little girl named Lily. She loved exploring her backyard and playing
Once upon a time, there lived a curious little girl named Lily. She loved exploring her backyard and playing
Once upon a time, there lived a curious little girl named Lily. She loved exploring her backyard and playing
Once upon a time, there lived a curious little girl named Lily. She loved exploring her backyard and playing
Once upon a time, there lived a curious little girl named Lily. She loved exploring her surroundings and learning
Once upon a time, there lived a curious little girl named Lily. She loved exploring her backyard and playing
Once upon a time, there lived a curious little girl named Lily. She loved exploring her surroundings and learning


## Calculate Loss Value

In [19]:
from jaxpt.dataloaders import  BlendedCloudDataLoader

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/alpha-448101-282bc1b884cd.json"

train_dl = BlendedCloudDataLoader(
    device_rank=1,
    block_size=2048,
    batch_size=16,
    bucket_names=["jaxpt_datasets", "jaxpt_datasets", "jaxpt_datasets"],
    bucket_prefixes=["smollm-corpus/processed/fineweb-edu-dedup",
    "smollm-corpus/processed/python-edu",
    "smollm-corpus/processed/cosmopedia-v2"],
    proportions=[87, 2, 11],
    label="train"
)

CloudDataLoader initialized:
------------------------
label:          train
shards:         1,936
shard size:     100,000,000
batch size:     13
block size:     2048
device rank:    1
------------------------
CloudDataLoader initialized:
------------------------
label:          train
shards:         40
shard size:     100,000,000
batch size:     2
block size:     2048
device rank:    1
------------------------
CloudDataLoader initialized:
------------------------
label:          train
shards:         279
shard size:     100,000,000
batch size:     1
block size:     2048
device rank:    1
------------------------


In [21]:
import optax

@nnx.jit
def loss_fn(model, batch, targets):
    logits = model(batch)
    loss = optax.softmax_cross_entropy_with_integer_labels(logits, targets)
    return loss.mean()

for i in range(50):
  batch, targets = train_dl()
  batch = batch.squeeze()
  targets = targets.squeeze()
  loss = loss_fn(m, batch, targets)
  print(loss)




3.1031368
3.081571
2.8853493
3.0571895
3.1564865
2.9638119
3.2175424
2.9653492
3.3352375
3.0586796
3.114626
3.1377487
3.1912823
3.4178452
3.714511
3.5787096
3.0736175
3.23954
3.238532
3.2009084
2.9266355
2.9748516
3.1112132
3.0605998
3.0871978
3.1140394
3.0846348
3.2620542
3.096765
2.96244
3.0412993
2.9762406
3.0503232
3.1516657
3.0868058
3.2164385
3.3193107
2.9705007
3.3482852
3.180911
3.1217504
3.2539396
3.0040944
3.118345
3.1888685
3.2766633
3.0494373
3.1150794
3.031064
3.0771372
